In [68]:
import os
import json
import pandas as pd
import traceback
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from callback import TokenUsageHandler

In [245]:
my_api = "Enter you GEMINI API-KEY"

In [113]:

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(my_api)
    
# os.environ["GOOGLE_API_KEY"] = my_api

In [115]:
llm = ChatGoogleGenerativeAI(api_key=my_api, model="gemini-2.5-flash", temperature=0.5) #temperatue means according to values wo creative behave karta hai

In [116]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import AimCallbackHandler
import PyPDF2

In [216]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer of this question"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer of this question"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer of this question"
    },
    
}

In [217]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guidance.
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [218]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE)


In [219]:
quiz_chain = LLMChain(llm=llm, prompt = quiz_generation_prompt, output_key="quiz", verbose=True)

quiz_evaluation_prompt = PromptTemplate(input_variables=("subject", "quiz"), template=TEMPLATE)

review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [220]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text','number','subject','tone','response_json'], output_variables=["quiz", "review"], verbose=True)

In [221]:
file_path = "/Users/krishna/Documents/MCQ Generator/data.txt"

In [222]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [223]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer of this question"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer of this question"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer of this question"}}'

In [234]:
NUMBER = 10
SUBJECT = "machine learning"
TONE = "simple"

In [235]:
from langchain_google_genai import ChatGoogleGenerativeAI
from callback import TokenUsageHandler

# Step 1: Create handler
handler = TokenUsageHandler()

# Step 2: Initialize Gemini LLM with callback
llm = ChatGoogleGenerativeAI(
    api_key=my_api,   
    model="gemini-2.5-flash",
    callbacks=[handler],
)

response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory 

In [141]:
# # Step 3: Make a request
# response = llm.invoke(prompt_text)
# Step 4: Print response and token usage

# print("Response:", response.content)
# print("Total tokens:", handler.total_tokens)
# print("Prompt tokens:", handler.prompt_tokens)
# print("Completion tokens:", handler.completion_tokens)

langchain_core.messages.ai.AIMessage

In [236]:
quiz = response.get("quiz")

In [237]:
start = quiz.find("{")
end = quiz.rfind("}") + 1
quiz_json = quiz[start:end]

In [238]:
quiz = json.loads(quiz_json)

In [239]:
quiz

{'1': {'mcq': 'What is Machine Learning primarily concerned with, according to the text?',
  'options': {'a': 'Developing explicit instructions for specific tasks.',
   'b': 'Studying the history of artificial intelligence.',
   'c': 'Developing and studying statistical algorithms that learn from data.',
   'd': 'Creating physical robots to perform manual labor.'},
  'correct': 'c'},
 '2': {'mcq': 'Which subdiscipline within machine learning has significantly boosted the performance of neural networks?',
  'options': {'a': 'Predictive analytics',
   'b': 'Data mining',
   'c': 'Deep learning',
   'd': 'Exploratory data analysis'},
  'correct': 'c'},
 '3': {'mcq': 'Neural networks are identified as a class of what type of algorithms?',
  'options': {'a': 'Explicit instructions',
   'b': 'Statistical algorithms',
   'c': 'Business problems',
   'd': 'Theoretical frameworks'},
  'correct': 'b'},
 '4': {'mcq': 'Which of the following is NOT listed as an application field for Machine Learni

In [240]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct":correct})

In [241]:
quiz_table_data

[{'MCQ': 'What is Machine Learning primarily concerned with, according to the text?',
  'Choices': 'a: Developing explicit instructions for specific tasks. | b: Studying the history of artificial intelligence. | c: Developing and studying statistical algorithms that learn from data. | d: Creating physical robots to perform manual labor.',
  'Correct': 'c'},
 {'MCQ': 'Which subdiscipline within machine learning has significantly boosted the performance of neural networks?',
  'Choices': 'a: Predictive analytics | b: Data mining | c: Deep learning | d: Exploratory data analysis',
  'Correct': 'c'},
 {'MCQ': 'Neural networks are identified as a class of what type of algorithms?',
  'Choices': 'a: Explicit instructions | b: Statistical algorithms | c: Business problems | d: Theoretical frameworks',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT listed as an application field for Machine Learning in the text?',
  'Choices': 'a: Computer vision | b: Speech recognition | c: Web de

In [242]:
quiz = pd.DataFrame(quiz_table_data)

In [244]:
quiz.to_csv("machine_learning.csv", index=False)